<a href="https://colab.research.google.com/github/AmineSdk/RecommenderSystem/blob/main/Notebook/CF_Social_Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download CellPhones -f CellPhonesRating.csv

In [2]:
from sklearn import preprocessing
from scipy.sparse import csr_matrix
import tensorflow as tf 
import numpy as np
import tensorflow_hub as hub
import pandas as pd
import random
from keras import datasets, layers, models
from sklearn.model_selection import train_test_split
from progressbar import progressbar

In [3]:
DATA = ['/content/CellPhonesRating_50K_reviews.csv','yelp_rating_mat','yelp_user_mat']
MODEL_PATH = []
PATH = ''
rmse = tf.keras.metrics.RootMeanSquaredError()
precision = tf.keras.metrics.Precision()
METRICS = ['accuracy','mae',rmse,precision]

In [5]:
df = pd.read_csv('/content/CellPhonesRating.csv.zip')

In [6]:
df_sample = df[:35000]
df_sample.to_csv('/content/CellPhonesRating_50K_reviews.csv')

In [7]:
df_sample['productID'].nunique()

2233

In [8]:
df_sample['reviewerID'].nunique()

25780

In [ ]:
!unzip "/content/yelp-user-user-trust.zip" -d "/content/"
!unzip "/content/yelp-review-ratings.zip" -d "/content/"

In [9]:
def DatasetToUserItemDataFrame(dataframe,userID,itemID,rating):
  #Setting new item IDs from string to int 
  itemKeys = [] 
  i = 0
  for item in dataframe[itemID].value_counts(sort=False):
    temp = np.full((item),i)
    itemKeys = np.append(itemKeys,temp)
    i += 1

  #Setting new user IDs from string to int
  userKeysDic = {}
  userKeys = np.zeros((dataframe[userID].size))
  i = 0
  for user in dataframe[userID].unique():
    userKeysDic[user] = i
    i += 1
  i = 0
  for user in dataframe[userID]:
    userKeys[i] = userKeysDic[user]
    i += 1

  #Converting arrays from float to int 
  userKeys = userKeys.astype(int)
  itemKeys = itemKeys.astype(int)

  
  user_item = csr_matrix((dataframe[rating].values.astype(int),(userKeys,itemKeys))) #Creating sparse matrix
  user_item_matrix = user_item.toarray() #Converting sparse matrix into array
  df_user_item = pd.DataFrame(user_item_matrix,index = dataframe[userID].unique()  ,columns = dataframe[itemID].unique() ) 

  return df_user_item

In [10]:
def getUsersRatings(df,usersList): #returns df_rating containing only users that have friends 
  temp = df.copy()
  for user in list(df['userID']):
    if user not in usersList:
      temp = temp.drop(temp.loc[temp['userID'] == user].index)
  return temp

In [11]:
def genIndexColumn(df,min_index,max_index,min_column,max_column): #generates random int index and columns for a given df
  New_User_IDs = random.sample(range(min_index,max_index),df.index.size)
  New_Item_IDs = random.sample(range(min_column,max_column),df.columns.size)

  df_new = pd.DataFrame(df.to_numpy(),index = New_User_IDs,columns = New_Item_IDs)
  return df_new

In [12]:
def generateIDs(df,index,columns,min_index,max_index,min_column,max_column):
  users = df[index].unique()
  items = df[columns].unique()
  df_train = df.copy()

  New_User_IDs = random.sample(range(min_index,max_index),df[index].nunique())
  New_Item_IDs = random.sample(range(min_column,max_column),df[columns].nunique())
  i = 0
  for d in progressbar(users) :
    df_train[index].replace({d : New_User_IDs[i]}, inplace=True)
    i+=1

  i = 0
  for d in progressbar(items) :
    df_train[columns].replace({d : New_Item_IDs[i]}, inplace=True)
    i+=1
  return df_train

In [13]:
def processGmfData(df,index,column,min_user_id,max_user_id,min_item_id,max_item_id):
  
  
  #df.rename(columns = {'text': 'reviewText', 'stars': 'rating', 'business_id': 'itemID', 'user_id': 'userID'}, inplace = True)
  df.rename(columns = {'reviewerID':'userID', 'productID' : 'itemID'},inplace = True)
  df = generateIDs(df,index,column,min_user_id,max_user_id,min_item_id,max_item_id)
  print("== PREPROCESSING DATA ...")
  df = df.dropna(how='any',axis=0)
  df.drop_duplicates(subset =['itemID', 'userID'] , keep = 'first' , inplace = True)

  df['one'] = df['rating'].apply(lambda x: 1 if x==1 else 0)
  df['two'] = df['rating'].apply(lambda x: 1 if x==2 else 0)
  df['three'] = df['rating'].apply(lambda x: 1 if x==3 else 0)
  df['four'] = df['rating'].apply(lambda x: 1 if x==4 else 0)
  df['five'] = df['rating'].apply(lambda x: 1 if x==5 else 0)
  #df['six'] = df['rating'].apply(lambda x: 1 if x==6 else 0)
  print("== DATA PREPROCESSED ==")

  return df

In [14]:
#GMF
def getGMFmodel(num_users,num_items,SIZE_):

  input_userID = layers.Input(shape=[1], name='user_ID')
  input_itemID = layers.Input(shape=[1], name='item_ID')

  user_emb_GMF = layers.Embedding(num_users, SIZE_, name='user_emb_GMF')(input_userID)
  item_emb_GMF = layers.Embedding(num_items, SIZE_, name='item_emb_GMF')(input_itemID)

  u_GMF = layers.Flatten()(user_emb_GMF)
  i_GMF = layers.Flatten()(item_emb_GMF)

  dot_layer = layers.Multiply()([u_GMF, i_GMF])

  out_layer = layers.Dense(5, activation='softmax', name='output')(dot_layer)

  GMF = tf.keras.Model([input_userID, input_itemID], out_layer)
  
  return GMF


def user_item_ID_lists(userIDs,itemIDs):
  i = 0
  item_s = pd.Series()
  user_s = pd.Series()
  for item in progressbar(itemIDs):
    temp = pd.Series(userIDs)
    user_s = item_s.append(temp)
    temp = []
    temp = [item for user in userIDs]
    temp = pd.Series(temp)
    item_s = item_s.append(temp)

  return user_s,item_s


def Fill_Cf_Matrix(model,userList,itemList,userIDs,itemIDs):

  prediction = model.predict([userList,itemList],verbose = 1)
  i = 0
  row = []
  matrix = []
  print("pred done")
  del prediction
  
  for i in progressbar(range(userList.shape[0])):
    result = np.where(prediction[i] == np.amax(prediction[i]))[0][0] + 1
    row.append(result)
    if len(row) == userIDs.size:
      matrix.append(row)
      row = []
       
    i += 1

  matrix_arr = np.array(matrix)
  matrix_arr = matrix_arr.transpose()
  dataframe = pd.DataFrame(matrix_arr, index = userIDs, columns = itemIDs)
  
  return dataframe

def loadDataset(fileID):
  dataset = pd.read_csv(DATA[fileID])
  print("== FILE LOADED ==")
  return dataset

def saveDataframe(df,fileID):
  file_path = PATH+'GMF_filled_'+DATA[fileID]
  df.to_csv(file_path)

def genGMFmat(df,df_test,modelID,fileID,nbrE,lossF,OF,emb,filter=None,cb=None):

  # df_og = loadDataset(fileID)
  # df = processGmfData(df_og,'userID','itemID',1000,2000+df_og['reviewerID'].nunique(),60000,61000+df_og['productID'].nunique())
  
  x_train ,x_test,y_train,y_test = train_test_split(df_test[['userID','itemID']],df_test[['one','two','three','four','five']],test_size=0.2,stratify=df_test[['one','two','three','four','five']])
  model_trained = trainModel(modelID,nbrE,lossF,OF,[x_train['userID'],x_train['itemID']],y_train,maxUserID=df_test['userID'].max() + 1,maxItemID =df_test['itemID'].max() + 1,embed_size=emb)
  model_trained.evaluate([x_test['userID'],x_test['itemID']],y_test)

  df_mat_filled = FillSparseMat(model_trained,df_test,df)
  
  return df_mat_filled

def FillSparseMat(model,df_prepro,df_og):

  user_s,item_s = user_item_ID_lists(df_prepro['userID'].unique(),df_prepro['itemID'].unique())
  df_mat_filled = Fill_Cf_Matrix(model,user_s,item_s,df_og['userID'].unique(),df_og['itemID'].unique())

  return df_mat_filled


def trainModel(modelID,nbrEpochs,lossF,OF,x_train ,y_train ,mid_layer_ratio=None,nb_layers=None,maxUserID = None,maxItemID = None,embed_size = None,filter_size = None):
  
  if modelID =='BLCNN':
    model = getBLCNNmodel(embed_size,filter_size)
  if modelID == 'GMF':
    model = getGMFmodel(maxUserID,maxItemID,embed_size)
  elif modelID == 'S-AutoCF':
    model = getAutoCFmodel(x_train,mid_layer_ratio,nb_layers)
  elif modelID == 'SS-AutoCF':
    model = getSS_HAEmodel(x_train,mid_layer_ratio,nb_layers)

  model.compile(optimizer = OF,
                    loss = lossF,
                    metrics= METRICS)
  model.fit(x_train,y_train,epochs = nbrEpochs,validation_split=0.14)
  
  return model

def getAutoCFmodel(x_train,mid_layer_ratio,nb_layers):
  #mid_layer_ratio [0 - 1]
  layer_ratio =  mid_layer_ratio + nb_layers*0.1

  encoder_input = layers.Input(shape=(x_train.shape[1]),name='user_item')
  flat = layers.Flatten()(encoder_input)
  for i in range(nb_layers):
    hid_encoder = layers.Dense(layer_ratio*x_train.shape[1],activation="relu")(hid_encoder)
    layer_ratio -= 0.1
  hid_encoder(flat)
  encoder_output = layers.Dense(mid_layer_ratio*x_train.shape[1],activation="relu")(hid_encoder)
  for i in range(nb_layers):
    decoder_input = layers.Dense(layer_ratio*x_train.shape[1],activation="relu")(decoder_input)
    layer_ratio += 0.1
  decoder_input(encoder_output)
  decoder_output = layers.Dense(x_train.shape[1],activation="relu")(decoder_input)

  autoencoder = tf.keras.Model(inputs = encoder_input, outputs = decoder_output)

  return autoencoder

def getSS_HAEmodel(x_train,mid_layer_ratio,nb_layers):
  
  #mid_layer_ratio [0 - 1]
  layer_ratio =  mid_layer_ratio + nb_layers*0.1

  #Social_Autoencoder

  rating_input = layers.Input(shape=(x_train.shape[1]),name='user_item')
  social_input = layers.Input(shape=(x_train.shape[1]),name='user_user')

  flat_rating = layers.Flatten()(rating_input)
  flat_social = layers.Flatten()(social_input)

  #dropout = layers.Dropout(.2)(flat)
  SharedLayer_encoder = layers.Concatenate()([flat_rating,flat_social])
  for i in range(nb_layers):
    hid_encoder = layers.Dense(layer_ratio*x_train.shape[1],activation="relu")(hid_encoder)
    layer_ratio -= 0.1
  hid_encoder(SharedLayer_encoder)
  encoder_output = layers.Dense(mid_layer_ratio,activation="relu")(hid_encoder)
  for i in range(nb_layers):
    hid_decoder = layers.Dense(layer_ratio*x_train.shape[1],activation="relu")(hid_decoder)
    layer_ratio += 0.1
  hid_decoder(encoder_output)
  SharedLayer_decoder =  layers.Dense(df_mat_rating.shape[1]+df_mat_trust.shape[1],activation="relu")(hid_decoder)
  rating_decoded , social_decoded = tf.split(SharedLayer_decoder,[df_mat_rating.shape[1],df_mat_trust.shape[1]],1)

  rating_output = layers.Dense(df_mat_rating.shape[1],activation="relu",name='rating_output')(rating_decoded)
  social_output = layers.Dense(df_mat_trust.shape[1],activation="relu",name='social_output')(social_decoded)

  autoencoder = tf.keras.Model(inputs = [rating_input,social_input], outputs = [rating_output,social_output])

  return autoencoder

def createAutoCF(modelID,input_fileID,target_fileID,nbrEpochs,lossF,OF,mid_layer_ratio,nb_layers):

  sparseDf = loadDataset(input_fileID)
  df_mat_filled = loadDataset(target_fileID)
  df_mat = DatasetToUserItemDataFrame(sparseDf,'userID','itemID','rating')
  x_train,x_test,y_train,y_test = train_test_split(df_mat,df_mat_filled)
  model_trained = trainModel(modelID,nbrEpochs,lossF,OF,x_train,y_train,mid_layer_ratio,nb_layers)

  return model_trained , x_test, y_test

def createSS_HAE(modelID,input_rating_fileID,target_rating_fileID,trust_fileID,nbrEpochs,lossF,OF,mid_layer_ratio,nb_layers):

  sparseDf = loadDataset(input_rating_fileID)
  df_rating_filled = loadDataset(target_rating_fileID)
  df_trust_mat = loadDataset(trust_fileID)
  trust_users_list = list(df_trust_mat.index)
  df_rating_filled = getUsersRatings(df_rating_filled,trust_users_list) 
  df_rating_filled = orgDataframe(df_rating_filled,trust_users_list)

  #input & target for autoencoder training
  df_rating_mat = DatasetToUserItemDataFrame(sparseDf,'userID','itemID','rating')
  df_rating_mat_filled = orgMatDataframe(df_rating_mat_filled,trust_users_list)

  #data split
  x_rating_train,x_rating_test,y_rating_train,y_rating_test = train_test_split(df_rating_mat,df_rating_mat_filled)
  x_train,x_test = train_test_split(df_trust_mat)

  model_trained = trainModel(modelID,nbrEpochs,lossF,OF,[x_rating_train,x_train],[y_rating_train,x_train],mid_layer_ratio,nb_layers)

  return model_trained , [x_rating_test,y_rating_test] , x_test

def evaluateModel(model,x_test,y_test):
  model.evaluate(x_test,y_test) 

def orgDataframe(df_to_org,org_list):

  df_org = pd.DataFrame(columns = ['userID','itemID','rating','reviewText'])
  for user in org_list:
    df_temp = df_to_org[(df_to_org['userID'] == user )]
    df_org = pd.concat([df_org,df_temp])

  return df_org

def orgMatDataframe(df_to_org,org_list):
  
  df_org = pd.DataFrame(index = list(df_to_org.index),columns = list(df_to_org.columns))
  for user in org_list:
    df_org.loc[user] = list(df_to_org.loc[user])
  
  return df_org

def getItemsScore(userID,modelID,fileID):
  
  listeItemScore = pd.DataFrame()
  
  if modelID == 0:
    listItemScore = predSeCF(userID,modelID,fileID)   #CF
  elif modelID == 1:
    listItemScore = predSeCF(userID,modelID,fileID)   #CF + Sentiment
  elif modelID == 2:
    listItemScore = predSSeCF(userID,modelID,fileID)  #CF + Social
  elif modelID == 3:
    listItemScore = predSSeCF(userID,modelID,fileID)  #CF + Sentiment + Social

  return listeItemScore 

def getModel(modelID):
  model = tf.keras.models.load_model(MODEL_PATH[modelID])
  return model

def loadDataFrame(df):
  df_loaded = pd.read_csv(DATA[df])
  return  df_loaded

def predSeCF(userID,modelID,fileID):
  
  model = getModel(modelID)
  df_user_item_mat = pd.read_csv(DATA[fileID])
  SparseScoresVec = df_user_item_mat.loc[userID].to_numpy()
  listItemScores = df_user_item_mat.loc[userID].to_frame()
  listItemScores = listItemScores.reset_index()
  listItemScores.set_axis(['itemID','score'],axis='columns',inplace=True)
  PredScoresVec = model.predict(SparseScoresVec)
  listItemScores.replace(SparseScoresVec,PredScoresVec,inplace = True)

  return listItemScores

def predSSeCF(userID,modelID,ratings_fileID,trust_fileID):

  model = getModel(modelID)
  df_user_item_mat = loadDataset(fileID)
  df_user_user_mat = pd.read_csv(trust_fileID)
  SparseScoresVec = df_user_item_mat.loc[userID].to_numpy()
  SparseTrustVec = df_user_user_mat.loc[userID].to_numpy()
  listItemScores = df_user_item_mat.loc[userID].to_frame()
  listItemScores = listItemScores.reset_index()
  listItemScores.set_axis(['itemID','score'],axis='columns',inplace=True)
  PredScoresVec,SparseTrustVec = model.predict([SparseScoresVec,SparseTrustVec])
  listItemScores.replace(SparseScoresVec,PredScoresVec,inplace = True)

  return listItemScores
  

In [16]:
def datapreprocess(df_sample):

  df_sample = df_sample.dropna(how='any',axis=0)
  df_sample.rename(columns = {'reviewerID':'userID', 'productID' : 'itemID'},inplace = True)
  df_sample.drop_duplicates(subset =['itemID', 'userID'] , keep = 'first' , inplace = True)
  df_sample = generateIDs(df_sample,'userID','itemID',1000,2000+df_sample['userID'].nunique(),60000,61000+df_sample['itemID'].nunique())
  print("== PREPROCESSING DATA ...")
  

  df_sample['one'] = df_sample['rating'].apply(lambda x: 1 if x==1 else 0)
  df_sample['two'] = df_sample['rating'].apply(lambda x: 1 if x==2 else 0)
  df_sample['three'] = df_sample['rating'].apply(lambda x: 1 if x==3 else 0)
  df_sample['four'] = df_sample['rating'].apply(lambda x: 1 if x==4 else 0)
  df_sample['five'] = df_sample['rating'].apply(lambda x: 1 if x==5 else 0)
  #df['six'] = df['rating'].apply(lambda x: 1 if x==6 else 0)
  print("== DATA PREPROCESSED ==")

  return df_sample

In [14]:
df_sample.shape

(34827, 5)

In [13]:
df_sample = df_sample.dropna(how='any',axis=0)
df_sample.rename(columns = {'reviewerID':'userID', 'productID' : 'itemID'},inplace = True)
df_sample.drop_duplicates(subset =['itemID', 'userID'] , keep = 'first' , inplace = True)

In [17]:
df_test = datapreprocess(df_sample)

100% (25780 of 25780) |##################| Elapsed Time: 0:02:51 Time:  0:02:51
100% (2233 of 2233) |####################| Elapsed Time: 0:00:13 Time:  0:00:13


== PREPROCESSING DATA ...
== DATA PREPROCESSED ==


In [20]:
del df

In [18]:
df_mat_filled = genGMFmat(df_sample,df_test,'GMF',0,5,tf.keras.losses.CategoricalCrossentropy(),'adam',32)

Epoch 1/5
749/749 [==============================] - 9s 6ms/step - loss: 1.4596 - accuracy: 0.5187 - mae: 0.3027 - root_mean_squared_error: 0.3826 - precision: 0.0000e+00 - val_loss: 1.3706 - val_accuracy: 0.5153 - val_mae: 0.2874 - val_root_mean_squared_error: 0.3707 - val_precision: 0.0000e+00
Epoch 2/5
749/749 [==============================] - 4s 6ms/step - loss: 1.3088 - accuracy: 0.5193 - mae: 0.2735 - root_mean_squared_error: 0.3620 - precision: 0.9726 - val_loss: 1.3353 - val_accuracy: 0.5153 - val_mae: 0.2720 - val_root_mean_squared_error: 0.3658 - val_precision: 0.4914
Epoch 3/5
749/749 [==============================] - 5s 6ms/step - loss: 0.9407 - accuracy: 0.6084 - mae: 0.2140 - root_mean_squared_error: 0.3035 - precision: 0.9882 - val_loss: 1.3592 - val_accuracy: 0.5068 - val_mae: 0.2764 - val_root_mean_squared_error: 0.3694 - val_precision: 0.4870
Epoch 4/5
749/749 [==============================] - 5s 6ms/step - loss: 0.3939 - accuracy: 0.9283 - mae: 0.1092 - root_mean_

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
100% (2233 of 2233) |####################| Elapsed Time: 0:12:47 Time:  0:12:47


1798961/1798961 [==============================] - 2408s 1ms/step


  0% (16915 of 57566740) |               | Elapsed Time: 0:00:00 ETA:   0:11:21

pred done


100% (57566740 of 57566740) |############| Elapsed Time: 0:12:09 Time:  0:12:09


ValueError: ignored

In [ ]:
df_mat_filled.head()